# Explore with Sqlite databases

In [1]:
import sys
sys.path.append("../python/")
import pentoref.IO as IO
import sqlite3 as sqlite

In [2]:
# Create databases if required
if False:   # make True if you need to create the databases from the derived data
    for corpus_name in ["TAKE", "TAKECV", "PENTOCV"]:
        data_dir = "../../../pentoref/{0}_PENTOREF".format(corpus_name)
        dfwords, dfutts, dfrefs, dfscenes, dfactions = IO.convert_subcorpus_raw_data_to_dataframes(data_dir)
        IO.write_corpus_to_database("{0}.db".format(corpus_name),
                                    corpus_name, dfwords, dfutts, dfrefs, dfscenes, dfactions)

In [3]:
# Connect to database
CORPUS = "PENTOCV"
db = sqlite.connect("{0}.db".format(CORPUS))
cursor = db.cursor()
# get the table column header names
print("utts", [x[1] for x in cursor.execute("PRAGMA table_info(utts)")])
print("words", [x[1] for x in cursor.execute("PRAGMA table_info(words)")])
print("refs", [x[1] for x in cursor.execute("PRAGMA table_info(refs)")])
print("scenes", [x[1] for x in cursor.execute("PRAGMA table_info(scenes)")])
print("actions", [x[1] for x in cursor.execute("PRAGMA table_info(actions)")])

utts ['gameID', 'uttID', 'starttime', 'endtime', 'utt', 'utt_clean', 'role', 'speaker']
words ['gameID', 'uttID', 'position', 'word', 'lemma', 'tag']
refs ['refID', 'gameID', 'uttID', 'text', 'id', 'piece', 'location']
scenes ['timestampID', 'gameID', 'pieceID', 'position_global', 'position_x', 'position_y', 'shape', 'shape_distribution', 'shape_orientation', 'shape_skewness_horizontal', 'shape_skewness_vertical', 'shape_edges', 'colour', 'colour_distribution', 'colour_hsv', 'colour_rgb']
actions ['gameID', 'starttime', 'endtime', 'hand', 'action', 'piece']


## Get utterances from certain time periods in each experiment or for certain episodes

In [4]:
for row in db.execute("SELECT gameID, starttime, speaker, utt_clean FROM utts" + \
                    " WHERE starttime >= 200 AND starttime <= 300" + \
                     ' AND gameID = "r8_1_1_b"' + \
                    " ORDER BY gameID, starttime"):
    print(row)

('r8_1_1_b', 203.325, 'A', 'Dieses?')
('r8_1_1_b', 203.83, 'A', '')
('r8_1_1_b', 203.879, 'B', 'Einmal ge- drehen')
('r8_1_1_b', 204.243, 'B', 'Genau')
('r8_1_1_b', 204.798, 'B', '')
('r8_1_1_b', 204.935, 'A', 'So?')
('r8_1_1_b', 205.317, 'A', '')
('r8_1_1_b', 205.412, 'B', 'Ja noch einmal drehen')
('r8_1_1_b', 206.449, 'B', '')
('r8_1_1_b', 207.158, 'B', 'Und unten daran')
('r8_1_1_b', 207.881, 'B', '')
('r8_1_1_b', 208.227, 'A', 'So?')
('r8_1_1_b', 208.732, 'A', '')
('r8_1_1_b', 208.882, 'B', 'Genau')
('r8_1_1_b', 209.523, 'B', '')
('r8_1_1_b', 210.055, 'B', 'Dann nehmen wir dein L und das L schließt unten links')
('r8_1_1_b', 212.879, 'B', '')
('r8_1_1_b', 213.779, 'B', 'Drehe es mal auf die andersherum')
('r8_1_1_b', 214.829, 'A', 'Ah okay so?')
('r8_1_1_b', 215.334, 'B', '')
('r8_1_1_b', 215.579, 'B', 'Genau')
('r8_1_1_b', 215.975, 'B', '')
('r8_1_1_b', 216.316, 'A', '')
('r8_1_1_b', 216.539, 'B', 'So und dann brauchen wir oben rechts den Klotz noch')
('r8_1_1_b', 218.367, 'B', ''

## Get mutual information between words used in referring expressions and properties of the referent

In [5]:
from collections import Counter
from pentoref.IOutils import clean_utt

In [6]:
piece_counter = Counter()
word_counter = Counter()
word_piece_counter = Counter()

for row in db.execute("SELECT id, gameID, text, uttID FROM refs"):
#for row in db.execute("SELECT shape, colour, orientation, gridPosition, gameID, pieceID FROM scenes"):
    #isTarget = db.execute('SELECT refID FROM refs WHERE gameID ="' + row[4] + '" AND pieceID ="' + row[5] + '"')
    #target = False 
    #for r1 in isTarget:
    #    target = True
    #if not target:
    #    continue
    #print(r)
    #shape, colour, orientation, gridPosition, gameID, pieceID = row
    #piece = gridPosition #shape + "_" + colour
    piece, gameID, text, uttID = row
    
    
    if CORPUS in ["TAKECV", "TAKE"]:
        for f in db.execute('SELECT word from words WHERE gameID ="' + str(gameID) + '"'):
            #print(f)
            for word in f[0].lower().split():
                word_counter[word] += 1
                word_piece_counter[piece+"__"+word]+=1
                piece_counter[piece] += 1
    elif CORPUS == "PENTOCV":
        for word in clean_utt(text.lower()).split():
            word_counter[word] += 1
            word_piece_counter[piece+"__"+word]+=1
            piece_counter[piece] += 1


In [7]:
good_pieces = ["X", "Y", "P", "N", "U", "F", "Z", "L", "T", "I", "W", "V", "UNK"]
print("non standard pieces", {k:v for k,v in piece_counter.items() if k not in good_pieces})
piece_counter

non standard pieces {'V,T': 2, 'P,T': 1, 'N,Z': 3, 'V,L': 3, 'T,Y': 3, 'W,T': 7, 'I,V,Z': 7}


Counter({'I': 477,
         'X': 493,
         'Y': 515,
         'W': 488,
         'P': 439,
         'N': 533,
         'U': 416,
         'F': 405,
         'Z': 383,
         'L': 498,
         'T': 517,
         'V': 482,
         'V,T': 2,
         'P,T': 1,
         'N,Z': 3,
         'V,L': 3,
         'T,Y': 3,
         'W,T': 7,
         'I,V,Z': 7,
         'UNK': 1})

In [8]:
word_counter.most_common(20)

[('das', 1371),
 ('den', 275),
 ('dem', 210),
 ('es', 206),
 ('blaue', 174),
 ('der', 166),
 ('t', 161),
 ('lila', 153),
 ('grüne', 143),
 ('stein', 136),
 ('kreuz', 122),
 ('orange', 113),
 ('z', 109),
 ('braune', 106),
 ('blauen', 98),
 ('teil', 84),
 ('gelbe', 83),
 ('winkel', 83),
 ('l', 83),
 ('rote', 82)]

In [9]:
word_total = sum(word_piece_counter.values())
piece_total= sum(piece_counter.values())

In [10]:
for piece, p_count in piece_counter.items():
    print("piece:", piece, p_count)
    p_piece = p_count/piece_total
    highest = -1
    best_word = ""
    rank = {}
    for word, w_count in word_counter.items():
        if w_count < 3: 
            continue
        p_word = w_count / word_total
        p_word_piece = word_piece_counter[piece+"__"+word] / word_total
        mi = (p_word_piece/(p_piece * p_word))
        rank[word] = mi
        if mi > highest:
            highest = mi
            best_word = word
    if True:
        top = 5
        for k, v in sorted(rank.items(), key=lambda x:x[1], reverse=True):
            print(k, v)
            top -=1
            if top <= 0: 
                break
    print("*" * 30)

piece: I 477
balken 11.89308176100629
dunkelblaue 11.89308176100629
ganz 11.89308176100629
lange 11.893081761006288
dunkelblauen 11.893081761006288
******************************
piece: X 493
kreuz 11.50709939148073
rot 11.50709939148073
roten 11.50709939148073
rotes 11.50709939148073
plus-zeichen 11.50709939148073
******************************
piece: Y 515
braune 11.015533980582525
braun 11.015533980582525
braunen 11.015533980582525
kleine 8.812427184466019
klötzchen 7.34368932038835
******************************
piece: W 488
hellgrüne 11.625
m 11.625
helle 11.625
treppchen 11.625
hellgrün 11.624999999999998
******************************
piece: P 439
pinken 12.92255125284738
pink 12.922551252847379
pinkfarbene 12.922551252847379
rosanen 12.922551252847379
pinke 12.747922181862958
******************************
piece: N 533
lilane 10.643527204502815
lilanen 10.643527204502815
lila 10.365265055365485
diesem 4.2574108818011265
dieses? 3.5478424015009384
******************************


In [11]:
db.close()